# Quick Start Example: Multi-Constraint Framework for Climate Sensitivity

This notebook demonstrates the basic usage of the multi-constraint framework for estimating equilibrium climate sensitivity (ECS).

## Overview

We will:
1. Load CMIP model data
2. Apply individual constraints (paleoclimate, observational, process-based)
3. Combine constraints using Bayesian framework
4. Validate results
5. Visualize the constrained ECS distribution

In [ ]:
# Import required packages
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.data_processing import ClimateDataProcessor
from src.constraints import (
    LGMConstraint,
    MPWPConstraint,
    HistoricalWarmingConstraint,
    EnergyBudgetConstraint,
    MultiConstraintFramework
)
from src.validation import PerfectModelTest, ConstraintIndependenceTest
from src.uncertainty import StructuralUncertaintyAnalysis

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Load CMIP Model Data

We'll load data from CMIP6 models. For this demonstration, we'll use synthetic data.

In [ ]:
# Initialize data processor
processor = ClimateDataProcessor()

# Load CMIP6 abrupt-4xCO2 experiment data
# This will generate synthetic data for demonstration
cmip_data = processor.load_cmip_data(
    experiment='abrupt-4xCO2',
    models=['CESM2', 'GFDL-CM4', 'IPSL-CM6A', 'UKESM1'],
    variables=['tas', 'rtnt']
)

print(f"Loaded data for {len(cmip_data.model)} models")
print(f"Variables: {list(cmip_data.data_vars)}")

# Display model ECS values (for synthetic data, these are in metadata)
print("\nModel ECS values:")
for model in cmip_data.model.values:
    ecs = cmip_data.sel(model=model).attrs.get('true_ecs', 'Unknown')
    print(f"  {model}: {ecs:.2f} K")

## 2. Apply Individual Constraints

Let's apply several constraints independently.

In [ ]:
# Initialize constraints
lgm_constraint = LGMConstraint()
hist_constraint = HistoricalWarmingConstraint()
energy_constraint = EnergyBudgetConstraint()

# Apply LGM constraint
print("Applying LGM constraint...")
lgm_result = lgm_constraint.apply(cmip_data)

# Apply historical warming constraint
print("Applying historical warming constraint...")
hist_result = hist_constraint.apply(cmip_data)

# Apply energy budget constraint
print("Applying energy budget constraint...")
energy_result = energy_constraint.apply(cmip_data)

print("\nAll constraints applied successfully!")

## 3. Combine Constraints

Now we'll use the multi-constraint framework to combine all constraints.

In [ ]:
# Create multi-constraint framework
mcf = MultiConstraintFramework(
    constraints=[lgm_constraint, hist_constraint, energy_constraint],
    method='bayesian',
    min_agreement=2
)

# Integrate constraints
combined_results = mcf.integrate_constraints(
    cmip_data=cmip_data,
    paleo_data=None,  # Will use defaults
    obs_data=None     # Will use defaults
)

print(f"\nCombined ECS estimate:")
print(f"  Mean: {combined_results['ecs_mean']:.2f} K")
print(f"  Median: {combined_results['ecs_median']:.2f} K")
print(f"  5-95% range: [{combined_results['ecs_5']:.2f}, {combined_results['ecs_95']:.2f}] K")
print(f"  17-83% range: [{combined_results['ecs_17']:.2f}, {combined_results['ecs_83']:.2f}] K")

## 4. Visualize Results

Let's create visualizations of the constrained ECS distribution.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Individual constraints
ax1 = axes[0]
for constraint_result in combined_results['individual_constraints']:
    ecs_samples = constraint_result['result']['ecs_constrained']
    ax1.hist(ecs_samples, bins=30, alpha=0.5, label=constraint_result['name'], density=True)

ax1.set_xlabel('ECS (K)', fontsize=12)
ax1.set_ylabel('Probability Density', fontsize=12)
ax1.set_title('Individual Constraint Results', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Combined distribution
ax2 = axes[1]
combined_ecs = combined_results['ecs_distribution']
ax2.hist(combined_ecs, bins=50, alpha=0.7, color='steelblue', density=True, label='Combined')

# Add vertical lines for percentiles
ax2.axvline(combined_results['ecs_5'], color='red', linestyle='--', label='5th percentile')
ax2.axvline(combined_results['ecs_median'], color='black', linestyle='-', linewidth=2, label='Median')
ax2.axvline(combined_results['ecs_95'], color='red', linestyle='--', label='95th percentile')

ax2.set_xlabel('ECS (K)', fontsize=12)
ax2.set_ylabel('Probability Density', fontsize=12)
ax2.set_title('Combined Constraint Distribution', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/ecs_constrained_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Validate Constraints

Let's perform validation tests to assess the reliability of our constraints.

In [ ]:
# Perfect model test
pm_test = PerfectModelTest()
pm_results = pm_test.run(
    constraints=[lgm_constraint, hist_constraint],
    cmip_data=cmip_data
)

print("Perfect Model Test Results:")
print(f"  Mean bias: {pm_results['summary']['mean_bias']:.3f} K")
print(f"  Mean RMSE: {pm_results['summary']['mean_rmse']:.3f} K")
print(f"  Fraction within 90% CI: {pm_results['summary']['fraction_within_90_ci']:.2%}")

In [ ]:
# Test constraint independence
ind_test = ConstraintIndependenceTest()
independence = ind_test.assess(combined_results['individual_constraints'])

print("\nConstraint Independence Test:")
print(f"  Mean absolute correlation: {independence['mean_absolute_correlation']:.3f}")
print(f"  Constraints appear independent: {independence['independent']}")

if independence['high_correlation_pairs']:
    print("\n  High correlation pairs:")
    for pair in independence['high_correlation_pairs']:
        print(f"    {pair['constraint_1']} <-> {pair['constraint_2']}: r = {pair['correlation']:.3f}")

## 6. Uncertainty Decomposition

Decompose the uncertainty into different components.

In [ ]:
# Structural uncertainty analysis
uncertainty_analysis = StructuralUncertaintyAnalysis()
uncertainty_decomp = uncertainty_analysis.decompose(combined_results)

print("\nUncertainty Decomposition:")
print(f"  Within-constraint uncertainty: {uncertainty_decomp['within_constraint_fraction']:.1%}")
print(f"  Between-constraint uncertainty: {uncertainty_decomp['between_constraint_fraction']:.1%}")

# Visualize uncertainty decomposition
fig, ax = plt.subplots(figsize=(8, 6))

labels = ['Within-Constraint', 'Between-Constraint']
sizes = [
    uncertainty_decomp['within_constraint_fraction'],
    uncertainty_decomp['between_constraint_fraction']
]
colors = ['#ff9999', '#66b3ff']

ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%',
       startangle=90, textprops={'fontsize': 12})
ax.set_title('Uncertainty Decomposition', fontsize=14, fontweight='bold')

plt.savefig('../results/uncertainty_decomposition.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Summary

Generate a summary report of the analysis.

In [ ]:
print("="*70)
print("MULTI-CONSTRAINT CLIMATE SENSITIVITY ANALYSIS SUMMARY")
print("="*70)

print("\n1. DATA:")
print(f"   - Number of CMIP models: {len(cmip_data.model)}")
print(f"   - Models: {', '.join([str(m.values) for m in cmip_data.model])}")

print("\n2. CONSTRAINTS APPLIED:")
for i, constraint_result in enumerate(combined_results['individual_constraints'], 1):
    diag = constraint_result['result']['diagnostics']
    print(f"   {i}. {constraint_result['name']}")
    print(f"      ECS: {diag['posterior_ecs_mean']:.2f} ± {diag['posterior_ecs_std']:.2f} K")

print("\n3. COMBINED ESTIMATE:")
print(f"   - Method: {combined_results['method']}")
print(f"   - Mean ECS: {combined_results['ecs_mean']:.2f} K")
print(f"   - Median ECS: {combined_results['ecs_median']:.2f} K")
print(f"   - Standard deviation: {combined_results['ecs_std']:.2f} K")
print(f"   - 5-95% credible interval: [{combined_results['ecs_5']:.2f}, {combined_results['ecs_95']:.2f}] K")
print(f"   - 17-83% credible interval: [{combined_results['ecs_17']:.2f}, {combined_results['ecs_83']:.2f}] K")

print("\n4. VALIDATION:")
print(f"   - Perfect model test bias: {pm_results['summary']['mean_bias']:.3f} K")
print(f"   - Perfect model test RMSE: {pm_results['summary']['mean_rmse']:.3f} K")
print(f"   - Coverage (90% CI): {pm_results['summary']['fraction_within_90_ci']:.1%}")
print(f"   - Constraints independent: {independence['independent']}")

print("\n5. UNCERTAINTY:")
print(f"   - Within-constraint: {uncertainty_decomp['within_constraint_fraction']:.1%}")
print(f"   - Between-constraint: {uncertainty_decomp['between_constraint_fraction']:.1%}")

print("\n" + "="*70)
print("\nAnalysis complete! Results saved to ../results/")

## Next Steps

- Load real CMIP6 data for production analysis
- Add additional constraints (MPWP, process-based)
- Perform sensitivity analysis
- Compare with IPCC AR6 estimates
- Generate publication-quality figures